<a href="https://colab.research.google.com/github/yvt-h/SCA_SOP/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Homework 2**

a. Recommend two features that potentially predict returns. Intuitively
explain why you expect these features to predict returns. You can present
the answer to this question in a text cell in the .ipynb file. Recommend
features that we did not use in class.



The first feature is **MarketCap/TA Ratio**. It gauges a company's market value relative to its assets. A high ratio suggests market optimism about future growth, potentially indicating higher future returns. Conversely, a low ratio may signal undervaluation, hinting at correction opportunities. This ratio serves as a barometer for market sentiment and growth expectations.

The second feature is **Operating Margin** measures profitability from core operations, indicating how well a company converts sales into profits. A robust operating margin suggests effective cost management and competitive advantage, traits of companies poised for growth. High or improving margins signal operational health and profit-generation capability, often translating to better stock performance.

b. Compute these features. Explain how your code avoids any look-ahead
bias.

In [ ]:
# Connecting the Python Code with the google drive to access the datasets
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing Necessary Python Libraries
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
from pandas import DateOffset

In [ ]:
#CRSP Data

# Importing CRSP price and returns datasets
Returns = pd.read_csv("/content/drive/MyDrive/MSBA_Quant/MonthlyRet_198001_202312csv.zip") #Importing Cleaned CRSP data

# Handling Missing values
Returns.PRC = abs(Returns.PRC)                                         # Converting Price Values to absolute numbers (CRSP sets PRC with a "-" symbol if it is comuted as bid-ask average when there is no actual trade)

# Market Cap Calculation
Returns['marketcap'] = Returns.SHROUT * Returns.PRC                    # Calculating Market Capitalization
Returns['marketcap'] = Returns.groupby('PERMNO')['marketcap'].shift()  # Lagged Market Capitalization
Returns['marketcap'] = Returns['marketcap'].apply(lambda x: 0.5 if x < 0.5 else x) #setting market cap to a min value of .5 because market_cap happend to be zero for some stocks, and b2m becomes unbounded

# Exchange Code Filters
exch_nyse_amex_Nasdaq = ['N', 'Q', 'A']
Returns = Returns[Returns.PRIMEXCH.isin(exch_nyse_amex_Nasdaq)].copy() #keeping only NYSE (N), AMEX(A) and Nasdaq (Q) stocks, ie. stocks listed on  US exchanges)

#Keep only ordinary common shares
ord_common_shares = [10, 11, 12]
Returns = Returns[Returns.SHRCD.isin(ord_common_shares)].copy()             #keeping only ordinary common shares - excludes unit trusts, ADRS, REITS, closed-end funds

# Minor Pre-processing
Returns.reset_index(inplace = True, drop = True)                                                # Reset Index

Returns = Returns[["PERMNO","PRIMEXCH","date","RET","PRC","SHROUT","marketcap","CFACSHR"]].copy() # Reordering the columns for clarity
Returns.RET = pd.to_numeric(Returns.RET, errors = 'coerce')                      #RET denoted missing value with alphanumeric values. convert it to Numeric with the 'coerce' option to set nonnumeric value to nan.

Returns.dropna(inplace = True)
#CRSP Data , prepare Date-time for merging with Compustat data

Returns["date"] = pd.to_datetime(Returns["date"])                       # Convert  "date" to a DateTime object
Returns["year"] = Returns["date"].dt.year                              # Extracting year
Returns["month"] = Returns["date"].dt.month                            # Extracting month


print("***********************************************************")
print("Returns dataframe from CRSP dataset")
print("***********************************************************")
count = Returns.groupby('date').count()

***********************************************************
Returns dataframe from CRSP dataset
***********************************************************


In [ ]:
# Date-time Manipulations
Returns.sort_values(by = ['PERMNO','date'], inplace = True)
#Compute two signals: 'new_issue' and 'ret_2_12'
Returns['SHROUT_adj'] = Returns.SHROUT*Returns.CFACSHR                            #adjust for stock splits before computing new issues. Why?
Returns['SHROUT_adj_lag12'] = Returns.groupby('PERMNO')['SHROUT_adj'].shift(12)
Returns['new_issue_asof_monthend'] =(Returns['SHROUT_adj'] - Returns['SHROUT_adj_lag12'])/Returns['SHROUT_adj_lag12']
Returns['new_issue'] = Returns.groupby('PERMNO')['new_issue_asof_monthend'].shift()                                 #Why shift ?

#compute compunded returns from month t-12 to t-2
Returns['ret_2_12'] = 1
for i in range(2,13):
    Returns['ret_2_12'] =  Returns['ret_2_12'] *  (1 + Returns.groupby('PERMNO')['RET'].shift(i))
#Returns
count = count = Returns.groupby('date').count()
count

,PERMNO,PRIMEXCH,RET,PRC,SHROUT,marketcap,CFACSHR,year,month,SHROUT_adj,SHROUT_adj_lag12,new_issue_asof_monthend,new_issue,ret_2_12
date,,,,,,,,,,,,,,
1980-02-29,4550,4550,4550,4550,4550,4550,4550,4550,4550,4550,0,0,0,0
1980-03-31,4551,4551,4551,4551,4551,4551,4551,4551,4551,4551,0,0,0,0
1980-04-30,4550,4550,4550,4550,4550,4550,4550,4550,4550,4550,0,0,0,0
1980-05-30,4548,4548,4548,4548,4548,4548,4548,4548,4548,4548,0,0,0,0
1980-06-30,4557,4557,4557,4557,4557,4557,4557,4557,4557,4557,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,5048,5048,5048,5048,5048,5048,5048,5048,5048,5048,4828,4828,4813,4828
2023-09-29,5032,5032,5032,5032,5032,5032,5032,5032,5032,5032,4816,4816,4791,4816
2023-10-31,5005,5005,5005,5005,5005,5005,5005,5005,5005,5005,4785,4785,4764,4785


In [ ]:
count = Returns.groupby('date').count()
count

,PERMNO,PRIMEXCH,RET,PRC,SHROUT,marketcap,CFACSHR,year,month,SHROUT_adj,SHROUT_adj_lag12,new_issue_asof_monthend,new_issue,ret_2_12
date,,,,,,,,,,,,,,
1980-02-29,4550,4550,4550,4550,4550,4550,4550,4550,4550,4550,0,0,0,0
1980-03-31,4551,4551,4551,4551,4551,4551,4551,4551,4551,4551,0,0,0,0
1980-04-30,4550,4550,4550,4550,4550,4550,4550,4550,4550,4550,0,0,0,0
1980-05-30,4548,4548,4548,4548,4548,4548,4548,4548,4548,4548,0,0,0,0
1980-06-30,4557,4557,4557,4557,4557,4557,4557,4557,4557,4557,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,5048,5048,5048,5048,5048,5048,5048,5048,5048,5048,4828,4828,4813,4828
2023-09-29,5032,5032,5032,5032,5032,5032,5032,5032,5032,5032,4816,4816,4791,4816
2023-10-31,5005,5005,5005,5005,5005,5005,5005,5005,5005,5005,4785,4785,4764,4785


In [ ]:
#Compustat data + merge
Cstat_data = pd.read_csv('/content/drive/MyDrive/MSBA_Quant/Compustatdata_for_signals_1980_2023.zip') #  cstat data
Cstat_data['lag_at'] = Cstat_data.groupby('LPERMNO')['at'].shift()                                 #assets the previous fiscal year
Cstat_data["date"] = pd.to_datetime(Cstat_data["datadate"], format="%Y%m%d")                      # "date" is set to DateTime object
Cstat_data['date'] = Cstat_data['date'].apply(lambda x: x + DateOffset(months=+5)) # Adding five months (using DataOffset library) assuming it takes at most 4 months for the data to reach the market

Cstat_data.sort_values(by = 'date', inplace = True)                                    # Sort by date
#Cstat_data["SeqNo_cstat"] = (Cstat_data['date']).rank(method = "dense").astype(int)
Cstat_data.rename(columns = {'LPERMNO' : 'PERMNO'}, inplace = True)
Returns.sort_values(by = 'date', inplace = True)
Cstat_data.sort_values(by = 'date', inplace = True)

merged_data = pd.merge_asof(Returns, Cstat_data, by = 'PERMNO', left_on = 'date', right_on = 'date', tolerance=dt.timedelta(days = 365))


In [ ]:
merged_data.head()

,PERMNO,PRIMEXCH,date,RET,PRC,SHROUT,marketcap,CFACSHR,year,month,...,oancf,oiadp,ppenb,sale,txdb,xad,xrd,xsga,costat,lag_at
0,64900,Q,1980-02-29,0.248148,16.750,366.0,4941.0,1.0000,1980,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54199,A,1980-02-29,-0.042683,19.625,1275.0,26137.5,12.0000,1980,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42550,N,1980-02-29,-0.063291,27.750,19128.0,566667.0,1.6000,1980,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31042,A,1980-02-29,-0.023810,41.000,4546.0,190932.0,1.1236,1980,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22541,N,1980-02-29,-0.246154,6.125,61264.0,497770.0,2.0000,1980,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_data['MarketCap_TA_Ratio'] = merged_data['marketcap'] / merged_data['at']

merged_data['OperatingMargin'] = merged_data['oiadp'] / merged_data['sale']

signals = ['MarketCap_TA_Ratio', 'OperatingMargin']
merged_data.dropna(subset = signals, how = 'any', inplace = True)               #Drop only if signals are na
merged_data = merged_data[merged_data.date.dt.year > 1989]

In [ ]:
count = merged_data.groupby(['year', 'month'])[['MarketCap_TA_Ratio', 'OperatingMargin']].count()
count.head(36)

MarketCap_TA_Ratio  OperatingMargin
year month                                     
1990 1                    4901             4901
     2                    4903             4903
     3                    4869             4869
     4                    4851             4851
     5                    5027             5027
     6                    4961             4961
     7                    4943             4943
     8                    4944             4944
     9                    4915             4915
     10                   4906             4906
     11                   4924             4924
     12                   4905             4905
1991 1                    4879             4879
     2                    4888             4888
     3                    4847             4847
     4                    4822             4822
     5                    4985             4985
     6                    4956             4956
     7                    4935             4935
     8                    4905             4905
     9                    4918             4918
     10                   4895             4895
     11                   4871             4871
     12                   4871             4871
1992 1                    4845             4845
     2                    4811             4811
     3                    4816             4816
     4                    4785             4785
     5                    4734             4734
     6                    4975             4975
     7                    4933             4933
     8                    4934             4934
     9                    4920             4920
     10                   4881             4881
     11                   4930             4930
     12                   4914             4914

In [ ]:
# Compute decile portfolio returns

merged_data['MarketCap_TA_Ratio_rank'] = merged_data.groupby(['year','month'])['MarketCap_TA_Ratio'].transform(lambda x: pd.qcut(x, 10, duplicates='drop',labels=False)) # Calculating Ranks based on Book to Market Value in the Cro
merged_data['OperatingMargin_rank'] = merged_data.groupby(['year','month'])['OperatingMargin'].transform(lambda x: pd.qcut(x, 10, duplicates='drop',labels=False)) # Calculating Ranks based on accruals Book to Market Value in the Cross-section

merged_data.reset_index(inplace = True, drop = True)              # Reset Index

merged_data


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PERMNO,PRIMEXCH,date,RET,PRC,SHROUT,marketcap,CFACSHR,year,month,...,txdb,xad,xrd,xsga,costat,lag_at,MarketCap_TA_Ratio,OperatingMargin,MarketCap_TA_Ratio_rank,OperatingMargin_rank
0,12257,Q,1990-01-31,-0.025974,4.68750,11862.0,5.708588e+04,1.00,1990,1,...,0.000,NaN,NaN,4.726,I,NaN,5291.608732,-0.233186,9,1.0
1,56119,N,1990-01-31,-0.069364,20.12500,7155.0,1.547269e+05,2.25,1990,1,...,6.946,NaN,NaN,91.168,I,141.036,1016.221750,0.063608,6,5.0
2,11339,Q,1990-01-31,-0.109375,14.25000,9209.0,1.473440e+05,1.00,1990,1,...,0.000,2.059,6.655,23.651,I,40.189,2403.222913,0.177396,8,8.0
3,62447,Q,1990-01-31,-0.175439,1.46875,10756.0,1.796925e+04,0.60,1990,1,...,0.036,NaN,3.473,15.232,I,31.647,530.097646,0.056026,4,4.0
4,11174,Q,1990-01-31,-0.132530,1.12500,3714.0,4.816612e+03,1.50,1990,1,...,0.075,0.007,NaN,0.965,A,4.412,734.351627,0.207949,5,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008321,19880,N,2023-12-29,0.248681,75.72000,78823.0,4.779827e+06,1.00,2023,12,...,571.401,NaN,NaN,507.437,A,7121.393,514.332671,0.021111,4,4.0
2008322,87487,Q,2023-12-29,0.110769,7.22000,386791.0,2.514142e+06,1.00,2023,12,...,21.258,NaN,NaN,NaN,A,3342.166,817.614703,0.084732,5,5.0
2008323,11174,Q,2023-12-29,-0.051333,75.77000,6383.0,5.098102e+05,1.00,2023,12,...,39.315,5.800,0.000,529.556,A,1145.312,358.912337,-0.108951,3,3.0
2008324,17879,Q,2023-12-29,-0.154811,0.60000,28814.0,2.045506e+04,1.00,2023,12,...,0.000,NaN,5.848,13.101,A,25.987,1558.243209,-97.522388,7,NaN


c. Compute combined scores based on these two features and form decile
portfolios. Let ‘diff’ be the portfolio that is long Decile 9 and short Decile 0. Compute the following for ‘diff’:

i. Average Returns and t-statistics

In [ ]:
#combine scores - ensure that bigger score implies better signal
merged_data['combined_pct_rank'] = merged_data['MarketCap_TA_Ratio_rank'] + merged_data['OperatingMargin_rank']
merged_data.dropna(subset = ['RET', 'combined_pct_rank'], how = 'any', inplace = True)   #Drop only observations where relevant variables are nan
merged_data = merged_data.loc[merged_data.year >= 2000].copy()
merged_data

,PERMNO,PRIMEXCH,date,RET,PRC,SHROUT,marketcap,CFACSHR,year,month,...,xad,xrd,xsga,costat,lag_at,MarketCap_TA_Ratio,OperatingMargin,MarketCap_TA_Ratio_rank,OperatingMargin_rank,combined_pct_rank
709529,10866,N,2000-01-31,-0.265487,10.375,18258.0,2.578942e+05,2.25,2000,1,...,54.9,NaN,551.877,A,694.988,393.592270,0.038543,3,3.0,6.0
709530,81015,Q,2000-01-31,0.294118,1.375,22641.0,2.405606e+04,1.00,2000,1,...,NaN,NaN,NaN,I,287.999,101.148142,0.075188,0,4.0,4.0
709531,76415,Q,2000-01-31,0.119534,6.000,3541.0,1.897756e+04,0.40,2000,1,...,NaN,NaN,NaN,I,22.463,1180.857730,0.009752,6,2.0,8.0
709532,71395,N,2000-01-31,0.037037,12.250,17878.0,2.111839e+05,1.00,2000,1,...,NaN,NaN,NaN,I,505.487,431.284999,0.037954,3,3.0,6.0
709533,72119,A,2000-01-31,0.187500,4.750,5104.0,2.041600e+04,1.00,2000,1,...,NaN,1.892,10.709,A,19.091,1193.987953,-0.271243,6,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008320,80362,Q,2023-12-29,0.184864,39.610,25125.0,8.399288e+05,1.00,2023,12,...,NaN,0.000,285.695,A,1937.428,406.121913,0.035875,3,4.0,7.0
2008321,19880,N,2023-12-29,0.248681,75.720,78823.0,4.779827e+06,1.00,2023,12,...,NaN,NaN,507.437,A,7121.393,514.332671,0.021111,4,4.0,8.0
2008322,87487,Q,2023-12-29,0.110769,7.220,386791.0,2.514142e+06,1.00,2023,12,...,NaN,NaN,NaN,A,3342.166,817.614703,0.084732,5,5.0,10.0
2008323,11174,Q,2023-12-29,-0.051333,75.770,6383.0,5.098102e+05,1.00,2023,12,...,5.8,0.000,529.556,A,1145.312,358.912337,-0.108951,3,3.0,6.0


In [ ]:
n_cut = 10
merged_data['combined_rank'] = merged_data.groupby(['year','month'])['combined_pct_rank'].transform(lambda x: pd.qcut(x, n_cut, duplicates='drop',labels=False)) # Calculating Ranks based on combined_pct_rank
meanret = merged_data.groupby(['year','month', 'combined_rank'])['RET'].mean().to_frame()   # Calculating average return for each decile (according to accrual ratio) for each month

In [ ]:
# Compute the difference between extreme portfolio returns and the Global mean for combined_pct_rank
meanret = meanret.unstack(level = -1).copy()                                       # Unstacking the grouped dataframe
meanret[('RET', 'diff')] = meanret[('RET', n_cut - 1)] -  meanret[('RET', 0)]              # Calculating the long short returns of the portfolio by substracting "rank 0" avg. return from "rank 9" avg. return

nmon = len(meanret)                                                                # nmon in number of months
meanret = meanret.stack(level = -1).copy()                                         # Stacking the dataframe to year-month index level

# Overall Portfolio Returns Statistics
global_mean_combined = meanret.groupby('combined_rank')['RET'].agg([np.mean, np.std])                # mean and standard deviation of regression coefficients
global_mean_combined['t-stat'] =np.sqrt(nmon - 1) *  global_mean_combined['mean']/global_mean_combined['std'] # t-statistics calculation
global_mean_combined

,mean,std,t-stat
combined_rank,,,
0,0.017159,0.098569,2.949170
1,0.011890,0.075278,2.675897
2,0.010585,0.066268,2.705931
3,0.010759,0.059791,3.048563
4,0.008704,0.057094,2.582554
5,0.007371,0.053994,2.312579
6,0.009196,0.052044,2.993534
7,0.009188,0.051556,3.019123
8,0.009019,0.051581,2.962286


ii. Market model α and β

In [ ]:
# Monthly Mean Portfolio Returns for accrual rank
asset_returns = merged_data.groupby(['year','month', 'combined_rank'])['RET'].mean().to_frame()   # Calculating average return for each decile (according to accrual ratio) for each month
meanret = asset_returns.unstack(level = -1).copy()                                       # Unstacking the grouped dataframe
meanret[('RET', 'diff')] = meanret[('RET', 9)] -  meanret[('RET', 0)]              # Calculating the long short returns of the portfolio by substracting "rank 0" avg. return from "rank 9" avg. return

nmon = len(meanret)                                                                # nmon in number of months
asset_returns = meanret.stack(level = -1).copy()                                         # Stacking the dataframe to year-month index level

In [ ]:
# Reset the index to convert the multi-level index into columns
asset_returns = asset_returns.reset_index()
# Rename columns
asset_returns.columns = ['year', 'month', 'rank', 'RET']
asset_returns

,year,month,rank,RET
0,2000,1,0,0.142585
1,2000,1,1,0.088455
2,2000,1,2,0.051441
3,2000,1,3,0.022180
4,2000,1,4,0.031478
...,...,...,...,...
2930,2023,12,4,0.142777
2931,2023,12,5,0.101127
2932,2023,12,6,0.075671
2933,2023,12,7,0.081941


In [ ]:
asset_returns.head(11)

,year,month,rank,RET
0,2000,1,0,0.142585
1,2000,1,1,0.088455
2,2000,1,2,0.051441
3,2000,1,3,0.022180
4,2000,1,4,0.031478
5,2000,1,5,0.003908
6,2000,1,6,0.005173
7,2000,1,7,0.000806
8,2000,1,8,-0.009367
9,2000,1,9,-0.013378


In [ ]:
import statsmodels.api as sm
# Importing CRSP price and returns datasets
Mkt_Risk_freeRate = pd.read_csv('/content/drive/MyDrive/MSBA_Quant/Market_Riskfree_2312.csv')     # read market returns and risk-free rate
Mkt_Risk_freeRate['mkt_excess_returns'] = Mkt_Risk_freeRate['Market'] - Mkt_Risk_freeRate['RiskfreeRate'] #compute market excess returns
asset_returns = pd.merge(asset_returns, Mkt_Risk_freeRate, how = "inner", on = ['year', 'month'])
asset_returns['excess_returns'] = asset_returns['RET'] - asset_returns['RiskfreeRate'] #compute asset excess returns
asset_returns.loc[asset_returns['rank'] == 'diff',  'excess_returns'] = asset_returns['RET']  #Long - short returns  for diff
asset_returns.head(11)

,year,month,rank,RET,Market,RiskfreeRate,mkt_excess_returns,excess_returns
0,2000,1,0,0.142585,-0.0433,0.0041,-0.0474,0.138485
1,2000,1,1,0.088455,-0.0433,0.0041,-0.0474,0.084355
2,2000,1,2,0.051441,-0.0433,0.0041,-0.0474,0.047341
3,2000,1,3,0.022180,-0.0433,0.0041,-0.0474,0.018080
4,2000,1,4,0.031478,-0.0433,0.0041,-0.0474,0.027378
5,2000,1,5,0.003908,-0.0433,0.0041,-0.0474,-0.000192
6,2000,1,6,0.005173,-0.0433,0.0041,-0.0474,0.001073
7,2000,1,7,0.000806,-0.0433,0.0041,-0.0474,-0.003294
8,2000,1,8,-0.009367,-0.0433,0.0041,-0.0474,-0.013467
9,2000,1,9,-0.013378,-0.0433,0.0041,-0.0474,-0.017478


In [ ]:
asset_returns['rank'] = asset_returns['rank'].astype(str)
ranks = set(asset_returns['rank'])
ranks_list = sorted([rank for rank in ranks if rank.isdigit()]) + ['diff']
ranks_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'diff']

In [ ]:
asset_returns['constant'] = 1 #add constant to the regression to get the intercept
for decile in ranks_list:                                                         #Loop through the date to pick one rank at a time and estimate alpha and beta for that portfolio
    x = asset_returns[['constant', 'mkt_excess_returns', ]].loc[asset_returns['rank'] == decile].copy()
    y = asset_returns['excess_returns'].loc[asset_returns['rank'] == decile].copy()
    model = sm.OLS(y,x)
    results = model.fit()
 #Print out the regression estimates for one portfolio at a time. Then print the corresponding t-statistics
    print('************************************************')
    print('Decile:' , decile)
    print('**************** estimates of alpha and beta ****************' )
    print(results.params) #gets the parameters                                  #The coefficient on the constant(also referred to as the 'intercept')) is alpha
                                                                                # The coefficient on 'mkt_excess_returns' is beta

    print('******** t-statistics for alpha and beta ***********' )              #Use the t-stats on the constant to test whether alpha is signficantly different from zero.
                                                                                # The coefficient on 'mkt_excess_returns' is beta
    print(results.tvalues)

************************************************
Decile: 0
**************** estimates of alpha and beta ****************
constant              0.007470
mkt_excess_returns    1.524432
dtype: float64
******** t-statistics for alpha and beta ***********
constant               1.819346
mkt_excess_returns    17.266077
dtype: float64
************************************************
Decile: 1
**************** estimates of alpha and beta ****************
constant              0.003274
mkt_excess_returns    1.328266
dtype: float64
******** t-statistics for alpha and beta ***********
constant               1.259450
mkt_excess_returns    23.765488
dtype: float64
************************************************
Decile: 2
**************** estimates of alpha and beta ****************
constant              0.00261
mkt_excess_returns    1.21081
dtype: float64
******** t-statistics for alpha and beta ***********
constant               1.231270
mkt_excess_returns    26.564459
dtype: float64
************